In [25]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

from transformers import AutoTokenizer
import transformers
import torch
import numpy as np
import scipy as sp
import shap

from utils import dump_jsonl, load_jsonl, set_random_seed
import torch
from transformers import AutoModelForSequenceClassification, AdamW, BertConfig

In [26]:
import pickle
def load_obj_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

In [27]:
import emoji
import pythainlp
from emoji import UNICODE_EMOJI

def is_emoji(s):
    for char in s:
        if char in UNICODE_EMOJI["en"]:
            return True
    return False

def print_lexicon(lexicons):
    s = ""
    for label in sorted(lexicons.keys()):
        s += "            "+label +" & "


        words = lexicons[label]
        sorted_words = dict(sorted(words.items(), key=lambda item: -item[1]))
        for k in list(sorted_words.keys())[0:20]:

#             unique = True
#             for l in lexicons:
#                 if l==label:
#                     continue

#                 if k in lexicons[l]:
#                     unique=False
                    
            unique = False
        
            if pythainlp.util.isthai(k):
                s += "\\thaitext{"+k+"}, "
            elif is_emoji(k):
                kk = ""
                for ch in k:
                    
                    ch = emoji.demojize(ch)
                    ch = ch.replace("_", "-").replace(":", "")
                    ch = "\emoji{"+ch+"}"
                    kk += ch
                print(kk)
                
                s += f"{kk}, "
            else:
                s += f"{k}, "
        s += "\\\\\n"

    s += "\hline"
#     print(s)
    return s

In [29]:
lexicons = load_jsonl("Lexicons/v2/lexicon_task1_clse.jsonl")[0]
s1 = print_lexicon(lexicons)

Loaded 1 records from Lexicons/v2/lexicon_task1_clse.jsonl


In [30]:
lexicons = load_jsonl("Lexicons/v2/lexicon_task1_auth.jsonl")[0]
s2 = print_lexicon(lexicons)

Loaded 1 records from Lexicons/v2/lexicon_task1_auth.jsonl
\emoji{smiling-face-with-smiling-eyes}
\emoji{face-with-tears-of-joy}\emoji{face-with-tears-of-joy}\emoji{face-with-tears-of-joy}
\emoji{face-with-tears-of-joy}\emoji{face-with-tears-of-joy}\emoji{face-with-tears-of-joy}


In [31]:
lexicons = load_jsonl("Lexicons/v2/lexicon_task2_clse.jsonl")[0]
s3 = print_lexicon(lexicons)

Loaded 1 records from Lexicons/v2/lexicon_task2_clse.jsonl
\emoji{rolling-on-the-floor-laughing}\emoji{rolling-on-the-floor-laughing}\emoji{rolling-on-the-floor-laughing}
\emoji{rolling-on-the-floor-laughing}\emoji{rolling-on-the-floor-laughing}\emoji{rolling-on-the-floor-laughing}


In [32]:
lexicons = load_jsonl("Lexicons/v2/lexicon_task2_auth.jsonl")[0]
s4 = print_lexicon(lexicons)

Loaded 1 records from Lexicons/v2/lexicon_task2_auth.jsonl
\emoji{smiling-face-with-heart-eyes}
\emoji{face-with-rolling-eyes}


In [33]:
lexicons = load_jsonl("Lexicons/v2/lexicon_task3_clse.jsonl")[0]
s5 = print_lexicon(lexicons)

Loaded 1 records from Lexicons/v2/lexicon_task3_clse.jsonl


In [34]:
lexicons = load_jsonl("Lexicons/v2/lexicon_task3_auth.jsonl")[0]
s6 = print_lexicon(lexicons)

Loaded 1 records from Lexicons/v2/lexicon_task3_auth.jsonl


In [35]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (s1, s2),
    (s3, s4),
    (s5, s6),
]

printed_text = ""
for section, (t1, t2) in zip(sections, table_contents):
    
    printed_text += "\subsection{"+section+"}"
    
    printed_text += '''
\\begin{longtable}[h]{
        p{\dimexpr 0.30\linewidth-2\\tabcolsep}
        p{\dimexpr 0.70\linewidth-2\\tabcolsep}
    }
        \hline
        Relationship Label  & Lexicons\\\\
        \hline
        \endfirsthead
        \endhead


        \multicolumn{2}{l}{\\textit{Closeness}} \\\\
'''
    printed_text += t1
    printed_text += '''
            \hline

        \multicolumn{2}{l}{\\textit{Respect}} \\\\
'''
    
    printed_text += t2
    printed_text += '''
            \hline
\end{longtable}
\n\n
'''
    
print(printed_text)

\subsection{Setting 1: Private Conversations with Self-Reported Labels}
\begin{longtable}[h]{
        p{\dimexpr 0.30\linewidth-2\tabcolsep}
        p{\dimexpr 0.70\linewidth-2\tabcolsep}
    }
        \hline
        Relationship Label  & Lexicons\\
        \hline
        \endfirsthead
        \endhead


        \multicolumn{2}{l}{\textit{Closeness}} \\
            1. Close & \thaitext{เตี่ย}, \thaitext{ประสบการณ์}, \thaitext{ฟาง}, \thaitext{เป็นไร}, \thaitext{เกมส์}, \thaitext{ภาษา}, \thaitext{แนะ}, \thaitext{ลอก}, \thaitext{โฆษณา}, \thaitext{ที่รัก}, \thaitext{เต}, \thaitext{เมีย}, \thaitext{หรือไม่}, \thaitext{ลมแรง}, the, \thaitext{เชียว}, \thaitext{สามี}, \thaitext{ข้าวผัด}, \thaitext{หลัก}, \thaitext{ตรงไหน}, \\
            2. Know each other & \thaitext{นนท์}, \thaitext{พันธ์}, \thaitext{ถนัด}, 500, \thaitext{ล่า}, \thaitext{คับ}, \thaitext{ดูแลตัวเอง}, \thaitext{แนะ}, \thaitext{อาศัย}, \thaitext{ถ่าย}, \thaitext{ช่องทาง}, \thaitext{โบว์}, \thaitext{ย่อย}, \thaitext{ทันที}, \tha

In [137]:
# from scipy import stats
# import pandas as pd

# def cal_corr(shap_lexicons, weights, label_fn):
#     df = []
#     for label in shap_lexicons:
#         count = defaultdict(int)
#         for w in shap_lexicons[label]:
#             if w not in lexicons:
#                 continue

#             for t in lexicons[w]:
#                 count[t] += 1

#         l = label_fn(label)
#         A = []
#         B = []
#         for feat in count:
#             k = (l, feat)
#             if k in weights and weights[k]!="":
#                 A.append(weights[k])
#                 B.append(count[feat])

#         spm = stats.spearmanr(A, B)
#         df.append({
#             "label": label,
#             "N": len(A),
#             "spearman": spm.statistic,
#             "pvalue": spm.pvalue
#         })

#     return pd.DataFrame(df)

In [138]:
# shap_lexicons = load_jsonl("Classifier/lexicon_task1_clse.jsonl")[0]
# weights = load_obj_values("linear_weights_task1_clse.pkl")

# cal_corr(shap_lexicons, weights, closeness_to_eng1)